<a href="https://colab.research.google.com/github/looopic/ML2Project/blob/main/ML2Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visual text recognition model by Carlo Huser (husercar@students.zhaw.ch)


This model is used to extract the ingredients from an image of groceries. I want to use this model to build an app to detect allergies or intolerances on specific ingredients.

# Data
I was thinking of making my own dataset for this problem, but I quickly found out, that it's too time consuming to annotate every single picture on my own.
Therefore I searched the internet for a dataset I could use and found the TextOCR dataset on kaggle (https://www.kaggle.com/datasets/robikscube/textocr-text-extraction-from-images-dataset?select=annot.csv)

Please use one of the two options below to download the dataset to the /content-folder. For the kaggle-option, you need to download your api-json first.

In [ ]:
!gdown 1zg0ofaVhgnJQ4bmfLeo5MG1iAtoT2HnO
!unzip OCR_data.zip

In [ ]:
# upload your kaggle api json-file
from google.colab import files
files.upload()

!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



!kaggle datasets download -d robikscube/textocr-text-extraction-from-images-dataset
!unzip textocr-text-extraction-from-images-dataset.zip


# Preprocess Images

Due to the images being real photographs, there are multiple words on each image. I need to extract those words on their own, so that i can label that word.

ATTENTION! The whole preprocessing can last up to 10 hours! You can interrupt the preprocessing earlier if you want to. You can choose how many you'd like to preprocess by changing the num_interations variable.

In [2]:
import os
import csv
import cv2
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import re
import string

In [3]:
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [4]:
def preprocess(img):
	old_h,old_w=img.shape[0],img.shape[1]
  
	#Pad the height.
	#If height is less than 512 then pad to 512
	if old_h<512:
		to_pad=np.ones((512-old_h,old_w,3))*255
		img=np.concatenate((img,to_pad))
		new_height=512
	else:
	#If height >512 then pad to nearest 10.
		to_pad=np.ones((roundup(old_h)-old_h,old_w,3))*255
		img=np.concatenate((img,to_pad))
		new_height=roundup(old_h)

	#Pad the width.
	if old_w<512:
		to_pad=np.ones((new_height,512-old_w,3))*255
		img=np.concatenate((img,to_pad),axis=1)
		new_width=512
	else:
		to_pad=np.ones((new_height,roundup(old_w)-old_w,3))*255
		img=np.concatenate((img,to_pad),axis=1)
		new_width=roundup(old_w)-old_w
	return img

In [ ]:
# initialize arrays to store data and labels
images = []
labels = []

num_iterations = 2048

# Create directories to store the images
image_dir = '/content/preprocessed_images/'
os.makedirs(image_dir, exist_ok=True)

# remove all pictures in preprocessed
image_dir = '/content/preprocessed_images/'
file_list = os.listdir(image_dir)
for file_name in file_list:
    file_path = os.path.join(image_dir, file_name)
    os.remove(file_path)

# read annotations csv file
with open('/content/annot.csv','r') as csvfile:
  reader = csv.reader(csvfile)
  
  #skip header
  next(reader)

  iterations = 0

  for row in tqdm(reader):
    image_path = "/content/train_val_images/train_images/"+row[2]+".jpg"
    boundary = row[5].strip('[]').split(',')
    boundaryArray = [float(value) for value in boundary]
    boundaryXmin = int(min(boundaryArray[0], boundaryArray[2], boundaryArray[4], boundaryArray[6]))
    boundaryXmax = int(max(boundaryArray[0], boundaryArray[2], boundaryArray[4], boundaryArray[6]))
    boundaryYmin = int(min(boundaryArray[1], boundaryArray[3], boundaryArray[5], boundaryArray[7]))
    boundaryYmax = int(max(boundaryArray[1], boundaryArray[3], boundaryArray[5], boundaryArray[7]))
    text = row[4]
    name = row[0]

    # Check if the text contains non-Roman letters or numbers
    # Skips everything containing anything other than roman letters, roman digits, whitespaces and punctuation
    if not re.match(r'^[a-zA-Z0-9\s' + re.escape(string.punctuation) + ']+$', text):
            continue

    #load image
    image = cv2.imread(image_path)

    #extract text region
    text_region = image[boundaryYmin:boundaryYmax, boundaryXmin:boundaryXmax]

    #preprocess text region
    text_region = preprocess(text_region)

    # Generate a unique filename for each image
    image_filename = f"{name}.jpg"

    # Save the preprocessed image to disk
    image_path = os.path.join(image_dir, image_filename)
    cv2.imwrite(image_path, text_region)

    #append data to arrays
    images.append(image_filename)
    labels.append(text)

    if iterations == num_iterations:
            break
    iterations += 1


In [ ]:
#encode labels to numerical values
label_encoder = LabelEncoder()
label_encoder.fit(labels)
encoded_labels = label_encoder.transform(labels)

print("Length Labels:",len(encoded_labels))
print("Length Images:",len(images))

# Model

I'm following the tutorial on medium.com (https://medium.com/geekculture/building-a-complete-ocr-engine-from-scratch-in-python-be1fd184753b)
but due to my data not being compiled the same as the data this tutorial uses, I had to adjust practically the whole data preprocessing.

The training on colab will take up to 24 hours. This is due to the memory intensity of an OCR model. 

In [ ]:
import tensorflow as tf

policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

from keras.layers import *
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import Sequential
from keras import metrics
import random
import tensorflow as tf
import gc

batch_size = 64

In [8]:
def unet(pretrained_weights=None, input_size=(128, 128, 1)):
    num_classes = len(label_encoder.classes_)
    inputs = Input(input_size)
    conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)

    up7 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)

    up8 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)

    up9 = Conv2D(32, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    
    flatten = Flatten()(conv9)
    
    conv10 = Dense(num_classes, activation='softmax')(flatten)

    model = Model(inputs, conv10)

    model.compile(optimizer=Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    if pretrained_weights:
        model.load_weights(pretrained_weights)

    return model

model = unet()


mc = ModelCheckpoint('/content/drive/MyDrive/ML2/weightsCP.h5',save_weights_only=True, save_freq=1,verbose = 1)
ec = EarlyStopping(patience=3, restore_best_weights=True, monitor="loss",
)

In [9]:
# Create a dataset from image paths and labels
dataset = tf.data.Dataset.from_tensor_slices((images, encoded_labels))

# Define preprocessing and augmentation functions
def preprocess_fn(image_path, label):

  try:
    # Join path components
    image_path = '/content/preprocessed_images/' + image_path

    # Remove trailing '/'
    image_path = tf.strings.regex_replace(image_path, '/.jpg$', '.jpg')

    # Load and preprocess the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1)
    image = tf.image.resize_with_pad(image, target_height=128, target_width=128)
    
    return image, label
  except tf.errors.NotFoundError:
        # Skip the instance if the file is not found
    return None, None




# Apply preprocessing
dataset = dataset.map(preprocess_fn)
dataset = dataset.filter(lambda image, label: image is not None and label is not None)

# Shuffle and batch the dataset
dataset = dataset.shuffle(buffer_size=len(images))
dataset = dataset.batch(batch_size)

# Split into training and test datasets
train_dataset = dataset.take(int(0.9 * len(images)))
test_dataset = dataset.skip(int(0.9 * len(images)))


In [ ]:
epochs = 4

# Clear the memory after each epoch
def clear_memory():
    gc.collect()
    tf.keras.backend.clear_session()

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    # Train the model
    model.fit(
        train_dataset,
        epochs=1,
        validation_data=test_dataset,
        callbacks=[mc, ec]
    )

   # Clear the memory
    #clear_memory()

model.save('/content/drive/MyDrive/ML2/ocr_model.h5')

# Evaluation and Comparision

I was not able to commpare my model with a pretrained one due to the complexity of the models. I was not able to find a model that would work exactly like mine. They either could only recognize single letters or they would need an additional model.
I also tried using pytesseract, but I was not able to write a functioning evaluation code snippet for that.



In [ ]:
#load own model. If training isn't executed, it will download the model from google drive
if os.path.exists("/content/ocr_model.h5"):
  own_model=tf.keras.models.load_model("/content/ocr_model.h5")
else:
  !gdown 1-6q7l5TIbGhSrBzoYdZYGWPUdRWAQUiN
  own_model=tf.keras.models.load_model("/content/ocr_model_trained.h5")

In [ ]:
eval_own=own_model.evaluate(test_dataset)
print(f"Evaluation of my model: \n Test Loss: {eval_own[0]}\n Test Accuracy:{eval_own[1]}")

#Application of my model

In [ ]:
from PIL import Image
import io

def preprocess_uploaded(image):
  image = image.resize((128,128))
  image = np.array(image)
  image = image[:,:,:1]
  image = image / 255.0
  image = np.expand_dims(image, axis=0)
  return image

uploaded_file = list(files.upload().values())[0]
image = Image.open(io.BytesIO(uploaded_file))
preprocessed = preprocess_uploaded(image)
predicted = own_model.predict(preprocessed)
predicted = np.argmax(predicted, axis=1)
result = label_encoder.inverse_transform(predicted)[0]

print(result)